In [ ]:
# r"C:\Users\ungdu\Downloads\Chat_Mini\mini_data.csv"

In [16]:
import pandas as pd

# Hàm đọc file CSV
def load_csv(file_path):
    """
    Đọc dữ liệu từ file CSV và trả về DataFrame.
    """
    try:
        df = pd.read_csv(r"C:\Users\ungdu\Downloads\Chat_Mini\mini_data.csv")
        print("Dữ liệu được tải thành công!")
        return df
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return None

# Hàm tiền xử lý dữ liệu
def preprocess_data(df):
    """
    Tiền xử lý dữ liệu:
    - Loại bỏ các giá trị NULL.
    - Chuẩn hóa tên cột.
    - Loại bỏ dòng trùng lặp.
    """
    # Loại bỏ giá trị NULL
    df = df.dropna()
    
    # Chuẩn hóa tên cột
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
    # Loại bỏ dòng trùng lặp
    df = df.drop_duplicates()
    
    print("Tiền xử lý dữ liệu hoàn tất.")
    return df

# Hàm ghi dữ liệu đã xử lý ra file CSV
def save_csv(df, output_path):
    """
    Ghi dữ liệu đã xử lý ra file CSV.
    """
    try:
        df.to_csv(output_path, index=False)
        print(f"Dữ liệu đã được lưu tại {output_path}")
    except Exception as e:
        print(f"Lỗi khi ghi file CSV: {e}")

# Hàm chính để kiểm tra với dữ liệu nhỏ
def main():
    # Đường dẫn file input và output
    input_path = "mini_data.csv"
    output_path = "processed_data.csv"
    
    # Đọc file CSV
    df = load_csv(input_path)
    if df is not None:
        # Tiền xử lý dữ liệu
        processed_df = preprocess_data(df)
        
        # Hiển thị một số dòng dữ liệu đã xử lý
        print("Dữ liệu đã xử lý:")
        print(processed_df.head())
        
        # Ghi dữ liệu ra file
        save_csv(processed_df, output_path)

# Gọi hàm chính
if __name__ == "__main__":
    main()


Dữ liệu được tải thành công!
Tiền xử lý dữ liệu hoàn tất.
Dữ liệu đã xử lý:
                                             câu_hỏi  \
0  Trường Đại Học Công Nghệ Thông Tin DHQG TP.HCM...   
1  Trường (UIT) Đại Học Công Nghệ Thông Tin DHQG ...   

                                         câu_trả_lời  
0  Trường Đại Học Công Nghệ Thông Tin DHQG TP.HCM...  
1  Trường Đại học Công nghệ Thông tin (UIT) đã đạ...  
Dữ liệu đã được lưu tại processed_data.csv


In [17]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Hàm để thực hiện chunking
def split_text_into_chunks(text, threshold=0.3):
    """
    Phân đoạn văn bản dựa trên độ tương đồng ngữ nghĩa giữa các câu.
    - text: Văn bản đầu vào.
    - threshold: Ngưỡng tương đồng để quyết định bắt đầu một chunk mới.
    """
    # Tách văn bản thành các câu
    nltk.download("punkt", quiet=True)
    sentences = nltk.sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    if not sentences:
        return []
    
    # Tạo biểu diễn TF-IDF cho các câu
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()
    
    # Tính độ tương đồng ngữ nghĩa giữa các câu
    similarities = cosine_similarity(vectors)
    
    # Tạo các chunk dựa trên độ tương đồng
    chunks = [[sentences[0]]]
    for i in range(1, len(sentences)):
        sim_score = similarities[i - 1, i]
        if sim_score >= threshold:
            chunks[-1].append(sentences[i])
        else:
            chunks.append([sentences[i]])
    
    # Gộp các câu trong mỗi chunk thành đoạn hoàn chỉnh
    return [' '.join(chunk) for chunk in chunks]

# Hàm xử lý chunking trên DataFrame
def apply_chunking_to_dataframe(df, column, threshold=0.3):
    """
    Áp dụng chunking cho một cột trong DataFrame.
    - df: DataFrame đầu vào.
    - column: Tên cột cần áp dụng chunking.
    - threshold: Ngưỡng tương đồng cho chunking.
    """
    df["chunks"] = df[column].apply(lambda x: split_text_into_chunks(str(x), threshold))
    print("Chunking hoàn tất.")
    return df

# Tích hợp vào hàm chính
def main_chunking():
    input_path = "processed_data.csv"
    output_path = "chunked_data.csv"
    
    # Đọc dữ liệu đã xử lý
    df = pd.read_csv(input_path)
    
    # Áp dụng chunking
    if "câu_hỏi" in df.columns:
        chunked_df = apply_chunking_to_dataframe(df, column="câu_hỏi", threshold=0.3)
        
        # Hiển thị dữ liệu chunked
        print("Dữ liệu đã chunked:")
        print(chunked_df[["câu_hỏi", "chunks"]].head())
        
        # Lưu dữ liệu đã chunked ra file CSV
        chunked_df.to_csv(output_path, index=False)
        print(f"Dữ liệu đã được lưu tại {output_path}")
    else:
        print("Cột 'câu_hỏi' không tồn tại trong dữ liệu.")

# Gọi hàm chính
if __name__ == "__main__":
    main_chunking()

Chunking hoàn tất.
Dữ liệu đã chunked:
                                             câu_hỏi  \
0  Trường Đại Học Công Nghệ Thông Tin DHQG TP.HCM...   
1  Trường (UIT) Đại Học Công Nghệ Thông Tin DHQG ...   

                                              chunks  
0  [Trường Đại Học Công Nghệ Thông Tin DHQG TP.HC...  
1  [Trường (UIT) Đại Học Công Nghệ Thông Tin DHQG...  
Dữ liệu đã được lưu tại chunked_data.csv
